In [ ]:
import sqlite3
import os
import pandas as pd
import numpy as np
import re

In [ ]:
dump_path='/Users/xxxxx'
dump_database_name='plugin.db'


db = sqlite3.connect(os.path.join(dump_path, dump_database_name))
c = db.cursor()

pd.read_sql(f'SELECT category,count(id) FROM extensions group by category', con=db)
# total
len(pd.read_sql(f'SELECT id FROM extensions', con=db))
# with privacy practices
len(pd.read_sql(f'SELECT id, name, category, rate, rateCount, users, privacy, size FROM extensions where privacy != "not provided" and privacy != "None"', con=db))



In [ ]:
import f
from tqdm import tqdm
from multiprocessing import Pool
# some collected extensions after few days or hours may be no longer available anymore

pool = Pool(10)
vaild_id = pd.read_sql(f'SELECT id FROM extensions', con=db)['id'].to_list()
invaild_id = []
jobs = []

with Pool(processes=10) as p:
    with tqdm(vaild_id) as pbar:
        for _ in p.imap_unordered(f.f, vaild_id):
            pbar.update()
            if _ is not None:
                invaild_id.append(_)

# second times try
invaild_id2 = []
with Pool(processes=10) as p:
    with tqdm(invaild_id) as pbar:
        for _ in p.imap_unordered(f.f, invaild_id):
            pbar.update()
            if _ is not None:
                invaild_id2.append(_)
                
print(len(invaild_id2))

In [ ]:
from openpyxl import load_workbook
def write_excel(filename,sheetname,dataframe):
    book = load_workbook(filename)
    writer = pd.ExcelWriter(filename, engine='openpyxl') 
    writer.book = book

    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    dataframe.to_excel(writer, sheetname,index=False)

    writer.save()

# conver the collected number of users to integer
def user_convert(u):
    if '+' in u:
        return int(float(u.split('+')[0].strip().replace(',','')))
    else: return int(float(u.split(' ')[0].strip().replace(',','')))
# conver the collected extension size to integer
def size_convert(s):
    scaler = {'KiB': 1,'MiB': 1024}
    s = re.split('([-+]?\d+\.\d+)|([-+]?\d+)',s.strip())
    s = [r.strip() for r in s if r is not None and r.strip() != '']
    return float(s[0])*scaler[s[1]]

In [ ]:
# convert properties in privacy practices to 9-dimensional vectors
rows = pd.read_sql(f'SELECT id, name, category, rate, rateCount, users, privacy, size FROM extensions where privacy != "not provided" and privacy != "None" and id not in {tuple(invaild_id2)}', con=db)
rows['users'] = rows['users'].apply(user_convert)
rows['size']=rows['size'].apply(size_convert)
for i in rows.index:
    for pp in rows.at[i, "privacy"].split(';'):
        rows.at[i, f"{pp}"] = 1

rows = rows.fillna(0)

dt =rows.copy()
dt.drop(['id'], axis=1, inplace=True)
dt.drop(['name'], axis=1, inplace=True)
dt.drop(['category'], axis=1, inplace=True)
dt.drop(['privacy'], axis=1, inplace=True)
dt.drop(['rate'], axis=1, inplace=True)
dt.drop(['rateCount'], axis=1, inplace=True)
dt.drop(['users'], axis=1, inplace=True)
dt.drop(['size'], axis=1, inplace=True)

smart_array = dt.copy().values
smart_array[:,0] = smart_array[:,0].astype(str)
categorical_index = list([0])

c.close()
db.close()
print(categorical_index)
print(len(smart_array))
print(smart_array[0])

In [ ]:
# determine the optimal K value
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
model = KMeans()

visualizer1 = KElbowVisualizer(model, k=(2,12), n_jobs = -1) #metric='distortion'
visualizer2 = KElbowVisualizer(model, k=(2,12),metric='silhouette', n_jobs = -1) #metric='silhouette'

# check both for the best K
visualizer1.fit(smart_array)        # Fit the data to the visualizer
visualizer1.show()  

visualizer2.fit(smart_array)        # Fit the data to the visualizer
visualizer2.show()  

In [ ]:
import kmedoids
from sklearn.metrics.pairwise import euclidean_distances

euc = euclidean_distances(smart_array)
# check the loss

# fp = kmedoids.fasterpam(euc,5)
# print('FasterPAM Loss:', fp.loss)

fp1 = kmedoids.fastpam1(euc,5) # 5 is my optimal K value -choose yours
print('Fast PAM Loss:', fp1.loss)

# pam = kmedoids.pam(euc,5)
# print('pam Loss:', pam.loss)

# alt = kmedoids.alternating(euc,5)
# print('Alternating Loss:', alt.loss)

# to see the medoids of the clusters
print(sorted(fp1.medoids))

In [ ]:
# save the clustering results to excel
dfpam = rows.copy()

dfpam['labels'] = fp1.labels
dfpam.drop(['id'], axis=1, inplace=True)
dfpam.drop(['name'], axis=1, inplace=True)
dfpam.drop(['category'], axis=1, inplace=True)
dfpam.drop(['privacy'], axis=1, inplace=True)
dfpam.drop(['rate'], axis=1, inplace=True)
dfpam.drop(['rateCount'], axis=1, inplace=True)
dfpam.drop(['users'], axis=1, inplace=True)

dfpam.drop(['size'], axis=1, inplace=True)

first_column = dfpam.pop('labels')
dfpam.insert(0, 'labels', first_column)


pd.set_option('display.max_rows', 5000)
tmp = dfpam.groupby(dfpam.columns.tolist(),as_index=False).size()
# tmp = tmp.groupby(['labels','category']).apply(lambda x: x.sample(frac=1)).drop(['category'], axis=1).drop(['labels'], axis=1)
tmp = tmp.groupby(['labels','category']).apply(lambda x: x.sample(frac=1))
tmp.to_excel('./kmedoids2.xlsx',index=False,sheet_name='Sheet1')
    


In [ ]:
from sklearn.metrics.pairwise import euclidean_distances,manhattan_distances
import requests

# find extension with the min distance to the medoid
def idxByMinVal(data, subclu, ana):
    dist = [sum(x) for x in data]
    idxs = np.argwhere(dist == np.amin(dist))
    minIdx = {'idx':-1,'users':-1,'rate':-1.0,'rateCount':-1}
    
    # if the distance of extensions is same, we follow the order 
    # to ckeck # of user, rating score, and then # of raters
    for indx in idxs:
        if minIdx['users'] == subclu.iloc[indx[0]]['users']:
            if minIdx['rate'] == subclu.iloc[indx[0]]['rate']:
                if minIdx['rateCount'] == subclu.iloc[indx[0]]['rateCount']:
                    minIdx['idx'] = indx[0]
                    minIdx['users'] = subclu.iloc[indx[0]]['users']
                    minIdx['rate'] = subclu.iloc[indx[0]]['rate']
                    minIdx['rateCount'] = subclu.iloc[indx[0]]['rateCount']
                    
            elif minIdx['rate'] < subclu.iloc[indx[0]]['rate']:
                minIdx['idx'] = indx[0]
                minIdx['users'] = subclu.iloc[indx[0]]['users']
                minIdx['rate'] = subclu.iloc[indx[0]]['rate']
                minIdx['rateCount'] = subclu.iloc[indx[0]]['rateCount']
                
        elif minIdx['users'] < subclu.iloc[indx[0]]['users']:
            minIdx['idx'] = indx[0]
            minIdx['users'] = subclu.iloc[indx[0]]['users']
            minIdx['rate'] = subclu.iloc[indx[0]]['rate']
            minIdx['rateCount'] = subclu.iloc[indx[0]]['rateCount']
    return minIdx['idx']

def get_col_name(datarow): 
    return datarow.columns[datarow.eq(1).any()].to_list()

In [ ]:
extension = pd.DataFrame(columns=['labels','category','id','name','pp'])
b = pd.read_excel('./kmedoids2.xlsx','Sheet2')

for label in b['labels'].drop_duplicates().values:
    for cate in b.loc[b['labels']==label]['category'].drop_duplicates().values:
        sublbl = b.loc[b['labels']==label]
        subclu =  sublbl.loc[sublbl['category']==cate]
        
        idx = idxByMinVal(manhattan_distances(subclu.iloc[:,7:16].values),subclu,[label,cate])
        minSubRow = subclu.iloc[idx]
        
        rowId =minSubRow['id']
        if requests.get(f'https://chrome.google.com/webstore/detail/chegg-answers-free/{rowId}?hl=ca').status_code==200:
            subclu = subclu[subclu['id']!=rowId]
            
        val = pd.DataFrame({'labels':label,'category':cate,'id':minSubRow['id'],'name':minSubRow['name'],'pp':minSubRow[7:16].sum()}, index=[0])
        extension = pd.concat([extension, val], ignore_index=True, axis = 0)
        
extension.groupby(extension.columns.tolist(),as_index=False).size().drop(['size'], axis=1, inplace=True)
extension.groupby(['labels','pp']).apply(lambda x: x.sample(frac=1)).drop(['pp'], axis=1).drop(['labels'], axis=1)

# extension
write_excel('./kmedoids2.xlsx','extensions',extension)


In [ ]:
extension = pd.read_excel('./kmedoids2.xlsx',sheet_name='extensions')
# see the use of the privacy practices in each cluster
for label in extension['labels'].drop_duplicates().values:
    for lid in extension.loc[extension['labels']==label]['id']:
        print(label, extension.loc[extension['id']==lid]['pp'],get_col_name(b[b['id']==lid].iloc[:,7:16]))
    print('\n')

# Select the next consecutive extension with the min distance to the medoid if the selected extension is not capable.

In [ ]:
replaceId=['pgpidfocdapogajplhjofamgeboonmmj']

def idxByMinValExcept(data,replaceId):
    # print(data)
    # print([sum(x) for x in data])
    pp = manhattan_distances(data.iloc[:,7:16].values)
    dist = [sum(x) for x in pp]
    idxs = np.argwhere(dist == np.amin(dist))
    
    subclu = data
    minIdx = {'idx':-1,'users':-1,'rate':-1.0,'rateCount':-1}
    
    for indx in idxs:
       
        rowId = subclu.iloc[indx[0]]['id']
        if rowId not in invaild_id2 and rowId not in replaceId:
            if minIdx['users'] == subclu.iloc[indx[0]]['users']:
                if minIdx['rate'] == subclu.iloc[indx[0]]['rate']:
                    if minIdx['rateCount'] == subclu.iloc[indx[0]]['rateCount']:
                        minIdx['idx'] = indx[0]
                        minIdx['users'] = subclu.iloc[indx[0]]['users']
                        minIdx['rate'] = subclu.iloc[indx[0]]['rate']
                        minIdx['rateCount'] = subclu.iloc[indx[0]]['rateCount']

                elif minIdx['rate'] < subclu.iloc[indx[0]]['rate']:
                    minIdx['idx'] = indx[0]
                    minIdx['users'] = subclu.iloc[indx[0]]['users']
                    minIdx['rate'] = subclu.iloc[indx[0]]['rate']
                    minIdx['rateCount'] = subclu.iloc[indx[0]]['rateCount']

            elif minIdx['users'] < subclu.iloc[indx[0]]['users']:
                minIdx['idx'] = indx[0]
                minIdx['users'] = subclu.iloc[indx[0]]['users']
                minIdx['rate'] = subclu.iloc[indx[0]]['rate']
                minIdx['rateCount'] = subclu.iloc[indx[0]]['rateCount']
    return minIdx['idx']

extension = pd.DataFrame(columns=['labels','category','id','name','# pp'])

for label in b['labels'].drop_duplicates().values:
    for cate in b.loc[b['labels']==label]['category'].drop_duplicates().values:
        sublbl = b.loc[b['labels']==label]
        subclu =  sublbl.loc[sublbl['category']==cate]
        
        idx = idxByMinValExcept(subclu,replaceId)
        minSubRow = subclu.iloc[idx]
        val = pd.DataFrame({'labels':label,'category':cate,'id':minSubRow['id'],'name':minSubRow['name'],'# pp':minSubRow[7:16].sum()}, index=[0])
        extension = pd.concat([extension, val], ignore_index=True, axis = 0)

extension.groupby(extension.columns.tolist(),as_index=False).size().drop(['size'], axis=1, inplace=True)
extension.groupby(['labels','category','# pp']).apply(lambda x: x.sample(frac=1)).drop(['# pp'], axis=1).drop(['labels'], axis=1)

# write_excel('./kmedoids2.xlsx','replace',extension)
